## Monitoring ML Training Pipeline: Data Extraction

**Purpose**: This notebook focuses on extracting data from a PostgreSQL database and saving it locally.

**Note**: If you do not have data in PostgreSQL, you can skip this step and use preloaded data from the data folder.

**Objective**: The main objective is to build a classification model for loan eligibility, determining whether a loan should be approved or rejected.

**Data Sources**:

- customer_details
- credit_details
- loan_details

**Data Quality Challenges**:

- Duplication
- Date format

---

## Prerequisite Tools

To perform this task, you will need the following tools:

- PostgreSQL
- SQLAlchemy

---

## Queries

All the necessary SQL queries are conveniently stored in a single script, `src/queries.py`, for easy reference and execution.

---


In [1]:
CREATE_TEMP_TABLE_LOAN = """
    create temp table loan as (
        select 
            lower(t1.loan_id) loan_id,
            lower(t1.customer_id) customer_id,
            lower(t1.loan_status) loan_status,
            cast(concat(split_part(t1.application_time, '-', 2), '-', split_part(t1.application_time, '-', 1), '-', split_part(t1.application_time, '-', 3)) as timestamp) application_time,
            t1.current_loan_amount,
            lower(t1.term) term,
            t1.tax_liens,
            lower(t1.purpose) purpose,
            t1.no_of_properties 
        from (
            select 
                row_number() over(partition by loan_id order by application_time desc) rnk,
                ld.*
            from (
                select distinct * 
                from loan_details 
                where cast(concat(split_part(application_time, '-', 2), '-', split_part(application_time, '-', 1), '-', split_part(application_time, '-', 3)) as timestamp) between '{start_date}' and '{end_date}') ld
        ) t1
        where rnk=1
    );
"""

CREATE_TEMP_TABLE_CUSTOMER = """
    create temp table customer as (
        select t2.* 
        from (
            select customer_id, count(*) cnt from (select distinct * from customer_details) cd 
            group by customer_id 
        ) t1 
        join (select distinct * from customer_details) t2
        on t2.customer_id = t1.customer_id
        where t1.cnt=1
    );
"""

CREATE_TEMP_TABLE_CREDIT = """
    create temp table credit as (
        select t2.* 
        from (
            select customer_id, count(*) cnt from (select distinct * from credit_details) cd 
            group by customer_id 
        ) t1 
        join (select distinct * from credit_details) t2
        on t2.customer_id = t1.customer_id
        where t1.cnt=1
    );
"""

GET_DATA = """
    select 
        t1.loan_id, t1.customer_id, t1.loan_status, t1.application_time, t1.current_loan_amount, t1.term, t1.tax_liens, t1.purpose, t1.no_of_properties,
        lower(t2.home_ownership) home_ownership, t2.annual_income, lower(t2.years_in_current_job) years_in_current_job, t2.months_since_last_delinquent, t2.no_of_cars, t2.no_of_children,
        t3.credit_score, t3.monthly_debt, t3.years_of_credit_history, t3.no_of_open_accounts, t3.no_of_credit_problems, t3.current_credit_balance, t3.max_open_credit, t3.bankruptcies
    from loan t1
    left join customer t2
    on t2.customer_id = t1.customer_id
    left join credit t3
    on t3.customer_id = t2.customer_id
"""

## Getting the data

1. Create connection to the data -> available in script `src/helpers.py`

2. Create temporarly tables -> available in script `src/etl.py`

3. Get the data from joined temporarly tables -> available in script `src/etl.py`



In [ ]:
import datetime
import sys
import os
import json
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from importlib import reload

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'dags', 'src'))

# Make sure you have SQL Credentials.json stored in dags folder
import helpersa
import config
import queries

In [2]:
#### helpers.py methods ####

def save_dataset(df: pd.DataFrame, path: str):
    """
    Save data set.
    :param df: DataFrame - The data to be saved.
    :param path: str - The file path to save the data.
    :return: None
    """
    df.to_csv(path, index=False)  # Save DataFrame to a CSV file without including the index.
    print(f"[INFO] Dataset saved to {path}")  # Print a message confirming the dataset has been saved.


In [5]:
%pip install psycopg2
import psycopg2

  Using cached psycopg2-2.9.3-cp38-cp38-win_amd64.whl (1.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Load credentials from a JSON file.
credentials = json.load(open(config.PATH_TO_CREDENTIALS, 'r'))

# Create a PostgreSQL database engine using the loaded credentials.
engine = create_engine(f"postgresql://{credentials['user']}:{credentials['password']}@{credentials['host']}:{credentials['port']}/{credentials['database']}")


In [7]:
def extract_data(start_date: datetime.date, end_date: datetime.date = datetime.date.today()) -> pd.DataFrame:
    """
    Extracts data from the database and returns it as a pandas dataframe.
    Queries are to be defined in the `queries.py` file.
    :param start_date: start date of the data to be extracted
    :param end_date: end date of the data to be extracted (defaults to today)
    :return: pandas dataframe
    """
    assert start_date <= end_date, "start_date must be less than end_date"
    print("[INFO] Extracting data from the database since {0} to {1} ...".format(start_date, end_date))
    
    # Dropping temporary tables and creating them
    engine.execute(text("""drop table if exists customer;""").execution_options(autocommit=True))
    engine.execute(text(queries.CREATE_TEMP_TABLE_CUSTOMER).execution_options(autocommit=True))
    engine.execute(text("""drop table if exists loan;""").execution_options(autocommit=True))
    engine.execute(text(queries.CREATE_TEMP_TABLE_LOAN.format(start_date=start_date, end_date=end_date)).execution_options(autocommit=True))
    engine.execute(text("""drop table if exists credit;""").execution_options(autocommit=True))
    engine.execute(text(queries.CREATE_TEMP_TABLE_CREDIT).execution_options(autocommit=True))
    
    # Reading data from the database
    df = pd.read_sql(text(queries.GET_DATA), engine)
    return df

def collect_data(start_date: datetime.date, end_date: datetime.date = datetime.date.today(), job_id: str = None):
    """
    Collects data from the database and dumps them in the directory of raw data `config.PATH_DIR_DATA`.
    :param start_date: start date of the data to be extracted
    :param end_date: end date of the data to be extracted (defaults to today)
    :param job_id: job id of the data to be extracted
    """
    assert isinstance(start_date, datetime.date)
    assert isinstance(end_date, datetime.date)
    assert isinstance(job_id, str)
    assert start_date <= end_date
    size = 0

    # Extracting data
    df = extract_data(start_date, end_date)
    size = df.shape[0]
    
    # Generating the filename for the raw data
    filename = os.path.join(config.PATH_DIR_DATA, "raw", f"{job_id}_" + start_date.strftime("%Y-%m-%d") + "_" + end_date.strftime("%Y-%m-%d") + ".csv")
    
    # Saving the dataset to the generated filename
    helpers.save_dataset(df, filename)
    return filename


In [10]:
# Generate a unique job ID using a helper function.
job_id = helpers.generate_uuid()

# Define start and end dates for data collection.
start_date = datetime.date(2015, 1, 1)
end_date = datetime.date(2015, 5, 31)

# Print the generated job ID.
print("job Id:", job_id)

# Collect data within the specified date range and save it to a CSV file.
print(collect_data(start_date, end_date, job_id))


job Id: aa4c3eaadb02409281b589829e3c9370
[INFO] Extracting data from the database since 2015-06-01 to 2015-12-31 ...
[INFO] Dataset saved to ../dags/data\raw\aa4c3eaadb02409281b589829e3c9370_2015-06-01_2015-12-31.csv
../dags/data\raw\aa4c3eaadb02409281b589829e3c9370_2015-06-01_2015-12-31.csv


---